<a href="https://colab.research.google.com/github/kmongsil1105/colab_ipynb/blob/main/AI_PyTorch(Softmax%ED%9A%8C%EA%B7%80%EC%9D%98_%EB%B9%84%EC%9A%A9%ED%95%A8%EC%88%98%EA%B5%AC%ED%95%98%EA%B8%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F

torch.manual_seed(1)

# 파이토치로 소프트맥스의 비용 함수 구현하기 (로우-레벨)

In [ ]:
# 3개의 원소를 가진 벡터 텐서를 정의하고, 이 텐서를 통해 소프트맥스 함수를 이해해보겠습니다
z = torch.FloatTensor([1, 2, 3])

In [ ]:
# 이 텐서를 소프트맥스 함수의 입력으로 사용하고, 그 결과를 확인해보겠습니다.
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [ ]:
# 3개의 원소의 값이 0과 1사이의 값을 가지는 벡터로 변환된 것을 볼 수 있습니다. 이 원소들의 값의 합이 1인지 확인해보겠습니다.
hypothesis.sum()

tensor(1.)

In [ ]:
# 총 원소의 값의 합은 1입니다. 
# 이번에는 비용 함수를 직접 구현해보겠습니다. 우선 임의의 3 × 5 행렬의 크기를 가진 텐서를 만듭니다.
z = torch.rand(3, 5, requires_grad=True)

In [ ]:
# 이제 이 텐서에 대해서 소프트맥스 함수를 적용합니다. 
# 단, 각 샘플에 대해서 소프트맥스 함수를 적용하여야 하므로 
# 두번째 차원에 대해서 소프트맥스 함수를 적용한다는 의미에서 dim=1을 써줍니다. 
hypothesis = F.softmax(z, dim=1)
print(hypothesis)
hypothesis.sum()

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


tensor(3.0000, grad_fn=<SumBackward0>)

In [ ]:
# 이제 각 행의 원소들의 합은 1이 되는 텐서로 변환되었습니다. 
# 소프트맥스 함수의 출력값은 결국 예측값입니다. 
# 즉, 위 텐서는 3개의 샘플에 대해서 5개의 클래스 중 어떤 클래스가 정답인지를 예측한 결과입니다.

# 이제 각 샘플에 대해서 임의의 레이블을 만듭니다
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [ ]:
# 이제 각 레이블에 대해서 원-핫 인코딩을 수행합니다.

# 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis) 
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [ ]:
# 위의 연산에서 어떻게 원-핫 인코딩이 수행되었는지 보겠습니다. 
# 우선, torch.zeros_like(hypothesis)를 통해 모든 원소가 0의 값을 가진 3 × 5 텐서를 만듭니다. 
# 그리고 이 텐서는 y_one_hot에 저장이 된 상태입니다.

# 두번째 줄을 해석해봅시다. 
# y.unsqueeze(1)를 하면 (3,)의 크기를 가졌던 y 텐서는 (3 × 1) 텐서가 됩니다. 
# 즉, 다시 말해서 y.unsqueeze(1)의 결과는 아래와 같습니다.
print(y.unsqueeze(1))

tensor([[0],
        [2],
        [1]])


In [ ]:
# 그리고 scatter의 첫번째 인자로 dim=1에 대해서 수행하라고 알려주고, 
# 세번째 인자에 숫자 1을 넣어주므로서 두번째 인자인 y_unsqeeze(1)이 알려주는 위치에 숫자 1을 넣도록 합니다. 
# 앞서 텐서 조작하기 2챕터에서 연산 뒤에 _를 붙이면 In-place Operation (덮어쓰기 연산)임을 배운 바 있습니다. 
# 이에 따라서 y_one_hot의 최종 결과는 결국 아래와 같습니다.

print(y_one_hot)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [ ]:
# 이제 비용 함수 연산을 위한 재료들을 전부 손질했습니다. 
# 소프트맥스 회귀의 비용 함수는 다음과 같았습니다.

cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)
